In [1]:
import pytorch_lightning as L

from pytorch_lightning.callbacks import (
    EarlyStopping,
    LearningRateMonitor,
    ModelCheckpoint,
    ModelSummary,
)

import torch

from model.deepfont import DeepFont
from model.lightning_generate_callback import GenerateCallback
from model.lightning_wrappers import DeepFontAutoencoderWrapper, DeepFontWrapper

from dataset.dataset import FontWeightDataset, FWDataset
from dataset.transformations import *

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from model.configs import load_config
import numpy as np
import random 
import os 
import cv2 as cv


In [2]:
config = load_config("F:/FWC216/configs/adobe-vfr.yaml")
#config

In [3]:
# Load raw data
data_path = "data/"
folders = os.listdir(data_path)

img_paths = []

for folder in folders: 
    cpath = data_path + folder 

    images = os.listdir(cpath)
    for image in images: 
        a_path = "F:/FWC216/" + data_path + folder + "/" + image
        img_paths.append(a_path)

    

In [4]:
# Shuffle and split data

random.shuffle(img_paths) 

total_size = len(img_paths)
train_size = int(0.7*total_size)
test_size  = int(0.1*total_size) 
valid_size =  total_size - train_size - test_size

train_data = img_paths[:train_size]
test_data  = img_paths[train_size:train_size + test_size]
valid_data = img_paths[train_size + test_size:]

In [5]:
# t = T1()
# for path in train_data: 
#     img = cv.imread(path, cv.IMREAD_GRAYSCALE)
#     img1 = Ptrans(img)
#     img2 = t(image=img)['image']
#     print(img1.shape, img2.shape)
#     break

    


In [6]:
# t_img.shape
# cv.imshow("n", t_img)


In [7]:
# Torch dataset


train_dataset =  FWDataset(img_folder = train_data)
test_dataset  =  FWDataset(img_folder = test_data )
valid_dataset =  FWDataset(img_folder = valid_data)



In [8]:
# Data loader 
batch_size = 64
train_loader =  DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=  4, persistent_workers= True)

valid_loader =  DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers= 4, persistent_workers= True)

test_loader  =  DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers= 4, persistent_workers= True)

In [9]:
torch.set_float32_matmul_precision("high")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")



Device: cuda


In [10]:
# Define model

#Auto encoder 
ae = DeepFontAutoencoderWrapper()
checkpoint = torch.load("output_ae/last.ckpt")
ae.load_state_dict(checkpoint['state_dict'])
#DeepFont 

train_config = config["training"]

model = DeepFontWrapper(
            model=DeepFont(
                autoencoder= ae.autoencoder,
                num_classes= 4,
            ),
            num_classes= 4,
            learning_rate=train_config["learning_rate"],
        )



In [11]:
torch.cuda.empty_cache()

In [12]:
# Define trainer 

training_callbacks = [
        EarlyStopping(monitor="val_loss", mode="min"),
        LearningRateMonitor(logging_interval="step"),
        ModelCheckpoint(
            dirpath="./output_df",
            save_top_k=2,
            monitor="val_loss",
            filename="deepfont-{epoch:02d}-{val_loss:.4f}-{val_accuracy:.4f}",
            save_last=True,
        ),
        ModelSummary(-1),
    ]



trainer = L.Trainer(
        # If ever doing gradient clipping here, remember that this
        # has an effect on training: it will take likely twice as
        # long to get to the same accuracy!
        # gradient_clip_algorithm="norm",
        # gradient_clip_val=1.0,
    max_epochs=40,
    callbacks=training_callbacks,
)


trainer.fit(
    model=model,
    train_dataloaders=train_loader,
    val_dataloaders=valid_loader,
    ckpt_path= None,
)

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
f:\FWC\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:652: Checkpoint directory F:\FWC216\output_df exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name               | Type                | Params | Mode  | In sizes         | Out sizes       
----------------------------------------------------------------------------------------------------------
0  | model              | DeepFont            | 169 M  | train | [2, 1, 105, 105] | [2, 4]          
1  | model.ae_encoder   | Sequential          | 83.5 K | train | [2, 1, 105, 105] | [2, 128, 12, 12]
2  | model.ae_encoder.0 | Conv2d              | 9.3 K  | train | [2, 1, 105, 105] | [2, 64, 48, 48] 
3  | mode

Epoch 11: 100%|██████████| 438/438 [00:54<00:00,  8.06it/s, v_num=0, train_acc=1.000, train_loss=0.00319, train_f1=1.000, train_pre=1.000, train_rec=1.000, val_loss=0.0304, val_accuracy=0.989] 


In [13]:
trainer.test(model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 63/63 [00:03<00:00, 19.51it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9902499914169312
         test_f1            0.9902499914169312
        test_loss          0.032494693994522095
     test_precision         0.9902499914169312
       test_recall          0.9902499914169312
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.032494693994522095,
  'test_acc': 0.9902499914169312,
  'test_f1': 0.9902499914169312,
  'test_precision': 0.9902499914169312,
  'test_recall': 0.9902499914169312}]

In [14]:
# filepath = "fwc_model.onnx"
# model.to_onnx(file_path=filepath, export_params=True)